In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/optiver_real_vol')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')



Mounted at /content/drive
/bin/bash: line 1: nvidia-smi: command not found
Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:

import pandas as pd
import numpy as np
import os

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')

book_wap_log_returns_df =  pd.read_parquet('book_first_10_min_wap_log_returns.parquet')
book_wap_log_returns_df

In [ ]:
multivariate_garch_data = pd.DataFrame()

multivariate_garch_data['st_id'] = book_wap_log_returns_df['row_id'].apply(lambda x: int(x.split('-')[0]))
multivariate_garch_data['time_id'] = book_wap_log_returns_df['row_id'].apply(lambda x: int(x.split('-')[1]))
multivariate_garch_data['log_return_book_wap1'] = book_wap_log_returns_df['log_return']

multivariate_garch_data

,st_id,time_id,log_return_book_wap1
1,1,5,0.000247
2,1,5,-0.000252
3,1,5,0.000630
4,1,5,-0.000295
5,1,5,0.000269
...,...,...,...
1827296,99,32767,-0.000064
1827297,99,32767,0.000006
1827298,99,32767,-0.000033
1827299,99,32767,-0.000010


In [ ]:
#multivariate_garch_data.drop(columns=['log_return_wap1'],inplace=True)
multivariate_garch_data['log_return_book_wap1'] = book_wap_log_returns_df['log_return']
multivariate_garch_data


,st_id,time_id,log_return_book_wap1
1,1,5,0.000247
2,1,5,-0.000252
3,1,5,0.000630
4,1,5,-0.000295
5,1,5,0.000269
...,...,...,...
1827296,99,32767,-0.000064
1827297,99,32767,0.000006
1827298,99,32767,-0.000033
1827299,99,32767,-0.000010


In [ ]:
import glob
#### Find wap2 log returns

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def log_returns_per_time_id(file_path):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap2'] =(df_book_data['bid_price2'] * df_book_data['ask_size2']+df_book_data['ask_price2'] * df_book_data['bid_size2'])  / (
                                      df_book_data['bid_size2']+ df_book_data['ask_size2'])

    df_book_data['log_return_wap2'] = df_book_data.groupby(['time_id'])['wap2'].apply(log_return).values
    df_book_data = df_book_data[~df_book_data['log_return_wap2'].isnull()]
    stock_id = file_path.split('=')[1]
    df_book_data['row_id'] = df_book_data['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_book_data[['row_id','wap2','log_return_wap2']]


def past_log_returns_per_stock(list_file):
    df_wap2_log_rets = pd.DataFrame()
    for file in list_file:
        df_wap2_log_rets = pd.concat([df_wap2_log_rets,
                                     log_returns_per_time_id(file)])
    return df_wap2_log_rets

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# following code should be run on GPU/CoLab not CPU. Takes too long on CPU.
list_order_book_file_train = glob.glob('book_train.parquet/*')

df_wap2_log_rets_train = past_log_returns_per_stock(list_file=list_order_book_file_train,)

df_wap2_log_rets_train.to_parquet('book_first_10_min_wap2_log_returns.parquet')
df_wap2_log_rets_train


<ipython-input-5-0476b52acfa1>:12: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_book_data['log_return_wap2'] = df_book_data.groupby(['time_id'])['wap2'].apply(log_return).values
<ipython-input-5-0476b52acfa1>:12: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_book_data['log_return_wap2'] =

,row_id,wap2,log_return_wap2
1,1-5,1.000899,0.000026
2,1-5,1.000765,-0.000133
3,1-5,1.001403,0.000637
4,1-5,1.001083,-0.000320
5,1-5,1.000932,-0.000150
...,...,...,...
1827296,99-32767,0.999701,-0.000023
1827297,99-32767,0.999701,0.000000
1827298,99-32767,0.999679,-0.000022
1827299,99-32767,0.999656,-0.000023


In [ ]:
book_wap2_log_returns_df =  pd.read_parquet('book_first_10_min_wap2_log_returns.parquet')

multivariate_garch_data['log_return_book_wap2'] = book_wap2_log_returns_df['log_return_wap2']
multivariate_garch_data

,st_id,time_id,log_return_book_wap1,log_return_book_wap2
1,1,5,0.000247,0.000026
2,1,5,-0.000252,-0.000133
3,1,5,0.000630,0.000637
4,1,5,-0.000295,-0.000320
5,1,5,0.000269,-0.000150
...,...,...,...,...
1827296,99,32767,-0.000064,-0.000023
1827297,99,32767,0.000006,0.000000
1827298,99,32767,-0.000033,-0.000022
1827299,99,32767,-0.000010,-0.000023


In [ ]:
def first_10_min_time_id_cum_ret(r1):
    return np.log1p(np.expm1(r1.sum()))#, np.log1p(np.expm1(r2.sum()))

wap1_wap2_time_id_cum_ret_log_returns_df = multivariate_garch_data.groupby(by=['st_id','time_id'])['log_return_book_wap1','log_return_book_wap2'].agg(first_10_min_time_id_cum_ret)
#wap1_wap2_time_id_cum_ret_log_returns_df = wap1_wap2_time_id_cum_ret_log_returns_df.reset_index()
wap1_wap2_time_id_cum_ret_log_returns_df


<ipython-input-7-43a1542b32db>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  wap1_wap2_time_id_cum_ret_log_returns_df = multivariate_garch_data.groupby(by=['st_id','time_id'])['log_return_book_wap1','log_return_book_wap2'].agg(first_10_min_time_id_cum_ret)


log_return_book_wap1  log_return_book_wap2
st_id time_id                                            
0     5                    0.002292              0.002325
      11                   0.000360              0.000801
      16                  -0.002074             -0.001493
      31                  -0.002828             -0.002053
      62                  -0.000002             -0.000281
...                             ...                   ...
126   32751               -0.000527             -0.000960
      32753                0.004436              0.004829
      32758                0.001525              0.002525
      32763                0.000256              0.000804
      32767               -0.000727             -0.001364

[428932 rows x 2 columns]

In [ ]:
os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
trade_wap_log_returns_df =  pd.read_parquet('trade_first_10_min_wap_log_returns.parquet')
trade_wap_log_returns_df

# multivariate_garch_data['log_return_trade'] = trade_wap_log_returns_df['log_return']
# multivariate_garch_data

,row_id,price,log_return
1,10-5,1.000106,-0.000021
2,10-5,1.000084,-0.000021
3,10-5,1.000495,0.000410
4,10-5,1.000380,-0.000115
5,10-5,1.000746,0.000366
...,...,...,...
498571,99-32767,0.999932,0.000135
498572,99-32767,0.999932,0.000000
498573,99-32767,0.999797,-0.000135
498574,99-32767,0.999854,0.000057


In [ ]:
def first_10_min_time_id_cum_ret(r):
    return np.log1p(np.expm1(r.sum()))

trade_time_id_cum_ret_log_returns_df = trade_wap_log_returns_df.groupby(by=['row_id'])['log_return'].agg(first_10_min_time_id_cum_ret,)
trade_time_id_cum_ret_log_returns_df

row_id
0-1000    -0.001305
0-10000    0.000243
0-10005    0.002791
0-10017    0.005041
0-10030    0.000080
             ...   
99-9972    0.003844
99-9973    0.000467
99-9976   -0.009213
99-9988   -0.000158
99-9993    0.002159
Name: log_return, Length: 428852, dtype: float32

In [ ]:
trade_time_id_cum_ret_log_returns_df = pd.DataFrame(trade_time_id_cum_ret_log_returns_df).reset_index()


In [ ]:
trade_time_id_cum_ret_log_returns_df['st_id'] = trade_time_id_cum_ret_log_returns_df['row_id'].apply(lambda x: int(x.split('-')[0]))
trade_time_id_cum_ret_log_returns_df['time_id'] = trade_time_id_cum_ret_log_returns_df['row_id'].apply(lambda x: int(x.split('-')[1]))

trade_time_id_cum_ret_log_returns_df = trade_time_id_cum_ret_log_returns_df.drop(columns=['row_id'])

In [ ]:
####### merge book and trade cumulative log returns returns

merged_wap1_wap2_trade_cum_rets = pd.merge(wap1_wap2_time_id_cum_ret_log_returns_df,trade_time_id_cum_ret_log_returns_df, left_on=['st_id','time_id'], right_on=['st_id','time_id'], how='left').ffill().bfill()
merged_wap1_wap2_trade_cum_rets


,st_id,time_id,log_return_book_wap1,log_return_book_wap2,log_return
0,0,5,0.002292,0.002325,0.001341
1,0,11,0.000360,0.000801,0.000803
2,0,16,-0.002074,-0.001493,-0.002540
3,0,31,-0.002828,-0.002053,-0.002261
4,0,62,-0.000002,-0.000281,-0.000274
...,...,...,...,...,...
428927,126,32751,-0.000527,-0.000960,-0.001209
428928,126,32753,0.004436,0.004829,0.004858
428929,126,32758,0.001525,0.002525,0.001292
428930,126,32763,0.000256,0.000804,-0.000225


In [6]:
import pickle

########### REORDER time_ids in the correct order

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/liquidity_features')
# train_feat_df_reordered = pickle.load(open('train_feat_df_reordered.pkl','rb'))
# correct_time_id_order = train_feat_df_reordered[train_feat_df_reordered['stock_id']== 0]['time_id'].values


os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))


# reorder time_ids to correct order

def my_reorder_stock_in_df(df, time_ids_reordered):
    common_values = [value for value in time_ids_reordered if value in df['time_id'].values]
    df = df.set_index('time_id')
    df = df.reindex(common_values)
    df = df.reset_index()
    return df

# Assuming you have a dataframe called 'df' and an array of reordered time_ids called 'time_ids_reordered'
multivariate_garch_data_reordered = merged_wap1_wap2_trade_cum_rets.groupby('st_id').apply(my_reorder_stock_in_df, time_ids_reordered=correct_time_id_order).reset_index(drop=True)
multivariate_garch_data_reordered

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# .to_parquet('multivariate_garch_data_reordered.parquet')

NameError: name 'merged_wap1_wap2_trade_cum_rets' is not defined

In [ ]:
#multivariate_garch_data_reordered.rename(columns={'log_return':'log_return_trade'},inplace=True)
os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
multivariate_garch_data_reordered.to_parquet('multivariate_garch_data_reordered.parquet')

In [ ]:

# def transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, df,col_name ):
#     unique_test_time_ids = correct_time_id_order

#     log_return_df = pd.DataFrame(index=unique_test_time_ids, columns=unique_stock_ids)
#     for s in unique_stock_ids:
#         st_index = df['st_id'] == s
#         t_index = df['time_id'][st_index]
#         log_return_df.loc[t_index, s] = df[col_name][st_index].values
#     log_return_df = log_return_df.ffill().bfill()
#     return log_return_df

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# multivariate_garch_data_reordered = pd.read_parquet('multivariate_garch_data_reordered.parquet')
# multivariate_garch_data_reordered
# correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))


# unique_stock_ids = multivariate_garch_data_reordered['st_id'].unique()

# log_return_book_wap1_df = transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, multivariate_garch_data_reordered[['time_id','st_id','log_return_book_wap1']],'log_return_book_wap1' )
# log_return_book_wap2_df = transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, multivariate_garch_data_reordered[['time_id','st_id','log_return_book_wap2']],'log_return_book_wap2' )
# log_return_trade_df = transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, multivariate_garch_data_reordered[['time_id','st_id','log_return_trade']],'log_return_trade' )

# log_return_book_wap1_df = log_return_book_wap1_df.astype(float)
# log_return_book_wap2_df = log_return_book_wap2_df.astype(float)
# log_return_trade_df = log_return_trade_df.astype(float)

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# pickle.dump(log_return_book_wap1_df,open('log_return_book_wap1_df.pkl','wb'))
# pickle.dump(log_return_book_wap2_df,open('log_return_book_wap2_df.pkl','wb'))
# pickle.dump(log_return_trade_df,open('log_return_trade_df.pkl','wb'))


In [ ]:
twenty_min_volatitliy_df

,row_id,target,first_10_min_vol
0,0-5,0.004136,0.004499
1,0-11,0.001445,0.001204
2,0-16,0.002168,0.002369
3,0-31,0.002195,0.002574
4,0-62,0.001747,0.001894
...,...,...,...
428927,126-32751,0.003461,0.003691
428928,126-32753,0.003113,0.004104
428929,126-32758,0.004070,0.003117
428930,126-32763,0.003357,0.003661


In [ ]:
import pickle

def transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, df,col_name ):
    unique_test_time_ids = correct_time_id_order

    first_10_min_vol_df = pd.DataFrame(index=unique_test_time_ids, columns=unique_stock_ids)
    for s in unique_stock_ids:
        st_index = df['st_id'] == s
        t_index = df['time_id'][st_index]
        first_10_min_vol_df.loc[t_index, s] = df[col_name][st_index].values
    first_10_min_vol_df = first_10_min_vol_df.ffill().bfill()
    return first_10_min_vol_df


os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/optimus_GPU_data/data')
twenty_min_volatitliy_df = pd.read_parquet('20_min_volatility.parquet/20_min_volatility_all_stocks.parquet')

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))


os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
multivariate_garch_data_reordered = pd.read_parquet('multivariate_garch_data_reordered.parquet')

twenty_min_volatitliy_df['time_id'] = multivariate_garch_data_reordered['time_id']
twenty_min_volatitliy_df['st_id'] = multivariate_garch_data_reordered['st_id']

unique_stock_ids = twenty_min_volatitliy_df['st_id'].unique()

del multivariate_garch_data_reordered

first_10_min_wap1_vol_df = transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, twenty_min_volatitliy_df[['time_id','st_id','first_10_min_vol']],'first_10_min_vol' )

first_10_min_wap1_vol_df = first_10_min_wap1_vol_df.astype(float)

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
pickle.dump(first_10_min_wap1_vol_df,open('first_10_min_wap1_vol_df.pkl','wb'))



In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))


In [ ]:
import glob
import pandas as pd
import numpy as np

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/optimus_GPU_data/data')
subset_paths = glob.glob('trade_train.parquet/stock_id=*')

train_target = pd.read_csv('train.csv')

trade_price_real_vol_df = pd.DataFrame()

for path in subset_paths:
    print(path)
    st_id = int(path.split('/')[1].split('_')[1].split('=')[1])
    # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
    if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
      trade_train_st = pd.read_parquet(path)
      ## compute realized volality in the first 10 mins using trade execution price (instead of WAP)
      trade_train_st['log_ret_price'] = trade_train_st.groupby(by='time_id')['price'].apply(log_return).values
      trade_train_st = trade_train_st[~trade_train_st['log_ret_price'].isnull()]
      df_realized_vol_per_stock =  pd.DataFrame(trade_train_st.groupby(['time_id'])['log_ret_price'].agg(realized_volatility)).rename(columns={'log_ret_price':'trade_real_vol_price'})

      target_st =train_target[train_target['stock_id']==st_id] # use when not called above
      target_st.index = [target_st["time_id"]] # use when not called above
      common_time_id = np.intersect1d(df_realized_vol_per_stock.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
      realized_vol_price_n_target_vol_df = pd.DataFrame({'trade_real_vol_price':df_realized_vol_per_stock.loc[common_time_id,"trade_real_vol_price"].values , 'target_vol':target_st.loc[common_time_id,"target"].values})
      realized_vol_price_n_target_vol_df['st_id'] = np.repeat(st_id,len(common_time_id))
      realized_vol_price_n_target_vol_df['time_id'] = common_time_id

      trade_price_real_vol_df = pd.concat([trade_price_real_vol_df,realized_vol_price_n_target_vol_df])


In [ ]:
import pickle

def transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, df,col_name ):
    unique_test_time_ids = correct_time_id_order

    first_10_min_vol_df = pd.DataFrame(index=unique_test_time_ids, columns=unique_stock_ids)
    for s in unique_stock_ids:
        st_index = df['st_id'] == s
        t_index = df['time_id'][st_index]
        first_10_min_vol_df.loc[t_index, s] = df[col_name][st_index].values
    first_10_min_vol_df = first_10_min_vol_df.ffill().bfill()
    return first_10_min_vol_df



os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))


os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
multivariate_garch_data_reordered = pd.read_parquet('multivariate_garch_data_reordered.parquet')

unique_stock_ids = trade_price_real_vol_df['st_id'].unique()

del multivariate_garch_data_reordered

trade_price_real_vol_df = transform_vstack_to_columns(unique_stock_ids,correct_time_id_order, trade_price_real_vol_df[['time_id','st_id','trade_real_vol_price']],'trade_real_vol_price' )

trade_price_real_vol_df = trade_price_real_vol_df.astype(float)

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
pickle.dump(trade_price_real_vol_df,open('trade_price_real_vol_df.pkl','wb'))



In [ ]:
trade_price_real_vol_df

,90,105,75,67,76,41,39,78,88,2,...,72,68,84,20,69,63,120,48,94,7
4294,0.004497,0.004573,0.012072,0.002414,0.003734,0.002108,0.003937,0.006832,0.002243,0.002862,...,0.002844,0.003656,0.004281,0.002822,0.002174,0.005011,0.005639,0.002913,0.003733,0.001522
31984,0.001157,0.001267,0.001001,0.000647,0.000672,0.000719,0.000553,0.001007,0.000848,0.001066,...,0.001289,0.001645,0.000710,0.000736,0.000579,0.001912,0.001103,0.001805,0.000636,0.000519
31570,0.001511,0.000871,0.001810,0.000629,0.000724,0.001048,0.000914,0.001149,0.001184,0.000906,...,0.000682,0.000965,0.000923,0.000569,0.000627,0.001636,0.001302,0.000959,0.001433,0.000483
5666,0.002141,0.001021,0.003582,0.001523,0.001380,0.001035,0.000754,0.001901,0.001569,0.001050,...,0.001357,0.001277,0.001630,0.001290,0.001047,0.001711,0.002042,0.001274,0.001318,0.001341
29740,0.001989,0.001120,0.002664,0.000927,0.000909,0.000944,0.000628,0.001484,0.001528,0.001551,...,0.001396,0.001494,0.001267,0.000857,0.000758,0.001457,0.001054,0.001038,0.001103,0.001328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24913,0.002141,0.001690,0.001851,0.001702,0.001826,0.001210,0.000968,0.002851,0.001171,0.000882,...,0.001669,0.000898,0.002397,0.000974,0.001383,0.002626,0.001184,0.001397,0.002291,0.001202
15365,0.002246,0.001584,0.003299,0.001469,0.001765,0.001129,0.000890,0.002790,0.002481,0.001111,...,0.001466,0.000995,0.002289,0.000837,0.001193,0.002392,0.001372,0.001575,0.002130,0.001354
29316,0.003018,0.001689,0.002354,0.001442,0.001675,0.001418,0.001125,0.002350,0.001231,0.001083,...,0.001595,0.001289,0.002586,0.001139,0.001119,0.002825,0.001744,0.001596,0.002116,0.001976
32195,0.002493,0.001324,0.002337,0.001642,0.001721,0.001213,0.001005,0.002969,0.001921,0.000824,...,0.001987,0.001176,0.002492,0.000957,0.001206,0.002439,0.001535,0.001699,0.002051,0.001344
